In [1]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-hs01esyj
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-hs01esyj
  Cloning https://github.com/quantopian/pyfolio.git to /tmp/pip-install-7mruhwgj/pyfolio_1ef4ec36a8a641b18b87b8d7039b5578
  Running command git clone -q https://github.com/quantopian/pyfolio.git /tmp/pip-install-7mruhwgj/pyfolio_1ef4ec36a8a641b18b87b8d7039b5578
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-7mruhwgj/elegantrl_c47123fbb30a4738a0e0ed3696debe36
  Running command git clone -q https://github.com/AI4Finance-Foundation/ElegantRL.git /tmp/pip-install-7mruhwgj/elegantrl_c47123fbb30a4738a0e0ed3696debe36
     |████████████████████████████████| 234 kB 7.9 MB/s 
     |████████████████████████████████| 177 kB 59.5 MB/s 
     |██████████████████████

In [2]:
!pip install IbPy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.1 MB/s 
  Created wheel for IbPy2: filename=IbPy2-0.8.0-py3-none-any.whl size=60248 sha256=151a69804bd6618dccef169d11c4d70dbcd223a04016b7b41015bb1e36948075
  Stored in directory: /root/.cache/pip/wheels/63/3a/fc/1c780168b0c1e563bcb4fd4c43360ee926c7c8b32e06349229
Successfully built IbPy2


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/STECH/Reinforcement Learning1

/content/drive/MyDrive/STECH/Reinforcement Learning1


In [5]:
from finrl import config
from finrl import config_tickers
import os
import warnings
warnings.filterwarnings('ignore')

In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

# import sys
# sys.path.append("../FinRL-Library")

import itertools

In [7]:
df = pd.read_csv('datasets/transform.csv')

In [8]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.INDICATORS,
                    use_vix=True,
                    use_turbulence=True,
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (7309, 8)
Successfully added vix
Successfully added turbulence index


In [9]:
list_ticker = processed['tic'].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

In [10]:
processed_full = pd.DataFrame(combination, columns=['date','tic']).merge(processed, on=['date','tic'],how='left')

In [11]:
processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,1993-05-10,EURAUD,1.76307,1.77368,1.75904,1.75904,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,1.759040,1.759040,13.720000,0.000000
1,1993-05-10,USDAUD,1.41110,1.42720,1.41040,1.42370,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,1.423700,1.423700,13.720000,0.000000
2,1993-05-10,USDCAD,1.26830,1.27110,1.26610,1.27110,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,1.271100,1.271100,13.720000,0.000000
3,1993-05-10,USDCHF,1.42710,1.46090,1.42650,1.45860,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,1.458600,1.458600,13.720000,0.000000
4,1993-05-10,USDGBP,0.63840,0.65320,0.63800,0.65130,0.0,0.0,0.000000,1.772973,1.729867,0.000000,-66.666667,100.000000,0.651300,0.651300,13.720000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63589,2022-05-16,USDAUD,1.44100,1.45476,1.43234,1.43390,0.0,0.0,0.022582,1.470520,1.316685,58.366845,120.064722,19.295117,1.373169,1.376788,27.469999,13.566394
63590,2022-05-16,USDCAD,1.29163,1.29814,1.28362,1.28449,279373.0,0.0,0.008995,1.310764,1.243793,53.756782,82.904547,0.330439,1.268975,1.269934,27.469999,13.566394
63591,2022-05-16,USDCHF,1.00131,1.00643,1.00072,1.00176,148198.0,0.0,0.017009,1.013379,0.923431,73.113053,142.765555,73.491745,0.955353,0.940382,27.469999,13.566394
63592,2022-05-16,USDGBP,0.81549,0.81848,0.81113,0.81160,0.0,0.0,0.013895,0.831083,0.750746,66.969186,115.218993,47.222252,0.781715,0.764825,27.469999,13.566394


In [12]:
processed_full = processed_full[processed_full['date'].isin(processed['date'])]

In [13]:
processed_full = processed_full.sort_values(['date','tic'])

In [19]:
processed_full.to_csv('processed_full',index=False)